<h1 style="color:black" align="center">Настройки перед обработкой данных</h1>

**Jointplot**

In [1]:
def show_jointplot(x, y, df, kind='reg', font_scale=1.5):
    
    sns.set(font_scale=font_scale)

    grid = sns.jointplot(df[x], df[y], kind=kind)
    
    grid.fig.set_figwidth(10)
    grid.fig.set_figheight(10)

**The correlation matrix**

In [2]:
def show_corr_matrix(df, font_scale=1, cmap='PuBu'):

    plt.figure(figsize = (12, 8))

    sns.set(font_scale=font_scale)

    sns.heatmap(df.corr(), annot=True, fmt='.1f', square=True, linewidths=.5, cmap=cmap)
    plt.title('Correlation matrix')

### Подключение библиотек и скриптов

In [3]:
import numpy as np
import pandas as pd

import matplotlib.image as img
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter('ignore')

from pylab import rcParams


%matplotlib inline
%config InlineBackend.figure_format = 'svg'


rcParams['figure.figsize'] = 10, 5


import os
for dirname, _, filenames in os.walk('./'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./Data_preprocessing-Copy1.ipynb
./Data_preprocessing.ipynb
./Model.ipynb
./README.md
./.ipynb_checkpoints\Data_preprocessing-checkpoint.ipynb
./.ipynb_checkpoints\Data_preprocessing-Copy1-checkpoint.ipynb
./input\sample_submission.csv
./input\test.csv
./input\train.csv
./save_data\housing_prepared_test.csv
./save_data\housing_prepared_train.csv
./save_data\sample_submission.csv


### Описание данных:

* `Id` - идентификационный номер квартиры

* `DistrictId` - идентификационный номер района

* `Rooms` - количество комнат

* `Square` - площадь

* `LifeSquare` - жилая площадь

* `KitchenSquare` - площадь кухни

* `Floor` - этаж

* `HouseFloor` - количество этажей в доме

* `HouseYear` - год постройки дома

* `Ecology_1, Ecology_2, Ecology_3` - экологические показатели местности

* `Social_1, Social_2, Social_3` - социальные показатели местности

* `Healthcare_1, Helthcare_2` - показатели местности, связанные с охраной здоровья

* `Shops_1, Shops_2` - показатели, связанные с наличием магазинов, торговых центров

* `Price` - цена квартиры

**Пути к директориям и файлам**

In [4]:
LOAD_TRAIN_DATASET = 'input/train.csv'
LOAD_TEST_DATASET = 'input/test.csv'

SAVE_TEST_DATASET = 'save_data/housing_prepared_test.csv'
SAVE_TRAIN_DATASET = 'save_data/housing_prepared_train.csv'

<h1 style="color:black" align="center">Загрузка данных</h1>

In [5]:
train = pd.read_csv(LOAD_TRAIN_DATASET)
test = pd.read_csv(LOAD_TEST_DATASET)

# Объединение для обработки данных
combine = [train, test]
combine = pd.concat(combine, ignore_index=True)

### Вывод информации по данным

In [6]:
display(combine.head())
display(combine.shape)

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,NaN,0,11,B,184966.930730
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,B,B,46,10309,1,240.0,1,16,B,300009.450063
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B,175616.227217
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,B,B,35,5776,1,2078.0,2,4,B,150226.531644


(15000, 20)

### Обзор категориальных переменных

**Получение признаков type='object'**

In [7]:
combine_obj_features = combine.select_dtypes(include='object')
combine_obj_features.head()

,Ecology_2,Ecology_3,Shops_2
0,B,B,B
1,B,B,B
2,B,B,B
3,B,B,B
4,B,B,B


**Получение уникальных строк для столбцов `Ecology_2	Ecology_3	Shops_2`**

In [8]:
# Выбор уникальных значений из строк
Ecology_2 = pd.DataFrame(combine_obj_features['Ecology_2'].unique(), columns=['Ecology_2'])
Ecology_3 = pd.DataFrame(combine_obj_features['Ecology_3'].unique(), columns=['Ecology_3'])
Shops_2 = pd.DataFrame(combine_obj_features['Shops_2'].unique(), columns=['Shops_2'])

# Соединение данных в один DataFrame
pd.concat([Ecology_2, Ecology_3, Shops_2], axis = 1)

,Ecology_2,Ecology_3,Shops_2
0,B,B,B
1,A,A,A


При выборке уникальных строк, получили в каждом столбце строки с уникальными значениями **[`A` и `B`]**. Исходя из этого, данные требуется преобразовать к целочисленному типу данных *int8*.

Где,

**`A` = 0**

**`B` = 1**

### Замена наблюдений у признаков `Ecology_2, Ecology_3, Shops_2` к виду `A = 0, B = 1`

* **Преобразование type = 'object' к type = 'uint8'**

In [9]:
combine[['Ecology_2','Ecology_3', 'Shops_2']] = pd.get_dummies(combine[['Ecology_2','Ecology_3', 'Shops_2']], drop_first=True)

combine.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,1,1,33,7976,5,NaN,0,11,1,184966.930730
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,1,1,46,10309,1,240.0,1,16,1,300009.450063
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,1,1,34,7759,0,229.0,1,3,1,220925.908524
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,1,1,23,5735,3,1084.0,0,5,1,175616.227217
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,1,1,35,5776,1,2078.0,2,4,1,150226.531644


<h1 style="color:black" align="center">Обработка пропусков</h1>

In [10]:
combine.isnull().sum()

Id                  0
DistrictId          0
Rooms               0
Square              0
LifeSquare       3154
KitchenSquare       0
Floor               0
HouseFloor          0
HouseYear           0
Ecology_1           0
Ecology_2           0
Ecology_3           0
Social_1            0
Social_2            0
Social_3            0
Healthcare_1     7175
Helthcare_2         0
Shops_1             0
Shops_2             0
Price            5000
dtype: int64

`Healthcare_1` - Данный признак имеет большое количество пропусков $50%$, что делает замену пропусков практически бесполезной и данный признак лучше удалить. Удалить признак можно будет после визуализации данных и обучения модели;

`LifeSquare` - Можно произвести замену средним арифметическим значением, но лучше оценить закономерность с другими данными и применить другой способ замены пропусков;

`Price` - Обладает пропусками, так как мы произвели объединение наборов данных.

In [11]:
combine.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
count,15000.000000,15000.000000,15000.000000,15000.000000,11846.000000,15000.000000,15000.000000,15000.000000,1.500000e+04,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,7825.000000,15000.000000,15000.000000,15000.000000,10000.000000
mean,8393.136933,50.693600,1.897000,56.360350,36.851792,6.174467,8.561800,12.606600,3.321575e+03,0.119196,0.990333,0.971733,24.769267,5370.404933,8.113667,1144.162428,1.319467,4.235133,0.917533,214138.857399
std,4850.111673,43.786257,0.839229,20.423913,71.120183,24.016979,5.323113,6.780165,1.637078e+05,0.119371,0.097846,0.165739,17.532278,4013.364403,23.841946,1029.296278,1.489012,4.796545,0.275084,92872.293865
min,0.000000,0.000000,0.000000,1.136859,0.333490,0.000000,1.000000,0.000000,1.908000e+03,0.000000,0.000000,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000,0.000000,59174.778028
25%,4188.750000,20.000000,1.000000,41.814371,22.896130,1.000000,4.000000,9.000000,1.974000e+03,0.017647,1.000000,1.000000,6.000000,1564.000000,0.000000,350.000000,0.000000,1.000000,1.000000,153872.633942
50%,8378.500000,37.000000,2.000000,52.645757,32.822810,6.000000,7.000000,12.000000,1.977000e+03,0.075424,1.000000,1.000000,25.000000,5285.000000,2.000000,900.000000,1.000000,3.000000,1.000000,192269.644879
75%,12594.250000,75.000000,2.000000,66.036608,45.144624,9.000000,12.000000,17.000000,2.001000e+03,0.195781,1.000000,1.000000,36.000000,7287.000000,5.000000,1548.000000,2.000000,6.000000,1.000000,249135.462171
max,16798.000000,212.000000,19.000000,641.065193,7480.592129,2014.000000,78.000000,117.000000,2.005201e+07,0.521867,1.000000,1.000000,74.000000,19083.000000,141.000000,4849.000000,6.000000,23.000000,1.000000,633233.466570


### `LifeSquare` - жилая площадь

In [12]:
# Коэффициент домножения

LS_of_S = combine.loc[combine['LifeSquare'].notnull(), ['Square', 'LifeSquare']].head()

coef = (LS_of_S['LifeSquare'] / LS_of_S['Square']).mean()

**Производим замену**

In [13]:
combine.loc[(combine['LifeSquare'] < 5) | (combine['LifeSquare'].isnull()), 'LifeSquare'] = combine['Square'] * coef

### `Helthcare_1` - показатели местности, связанные с охраной здоровья

In [14]:
combine.loc[combine['Healthcare_1'].isnull(), 'Healthcare_1'] = np.round(combine["Healthcare_1"].mean())

### Проверка итоговых значений

In [15]:
combine.isnull().sum().sum()

5000

$5000$ пропусков находятся при признаке `Price`, которые предстоит ещё предсказать.